### De 2 största metoderna är content filtering och collaborative filtering.

In [2]:
# importera biblotek

import pandas as pd 
import sklearn as sk
import seaborn as sbn
import numpy as np
import matplotlib.pyplot as plt



In [3]:
# läs in flera csv filer 

df_links = pd.read_csv("/Users/paraugustsson/Machine-learning-AI24-1/data/links.csv") #innehåller korsrefernser mellan olika filmsajter kräver mer avancerade tekniker och dash för presentation

df_movies = pd.read_csv("/Users/paraugustsson/Machine-learning-AI24-1/data/movies.csv")

df_ratings = pd.read_csv("/Users/paraugustsson/Machine-learning-AI24-1/data/ratings.csv")

df_tags = pd.read_csv("/Users/paraugustsson/Machine-learning-AI24-1/data/tags.csv")


In [4]:
# Undersök df_movies 
df_eda_mov = df_movies

df_eda_mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  86537 non-null  int64 
 1   title    86537 non-null  object
 2   genres   86537 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [5]:
df_eda_mov.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [6]:
df_eda_mov

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),Action|Drama
86533,288971,Ouija Japan (2021),Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller


In [7]:
# ta bort onödiga tecken

#remove


In [8]:
# Visa alla rader där titeln förekommer mer än en gång
duplicates = df_movies[df_movies.duplicated(subset="title", keep=False)]
print(duplicates.info())

<class 'pandas.core.frame.DataFrame'>
Index: 409 entries, 580 to 86388
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  409 non-null    int64 
 1   title    409 non-null    object
 2   genres   409 non-null    object
dtypes: int64(1), object(2)
memory usage: 12.8+ KB
None


In [9]:
# Räkna antalet förekomster per titel
title_counts = df_movies['title'].value_counts()
# Filtrera ut de titlar som förekommer mer än en gång
duplicate_titles = title_counts[title_counts > 1]
print(duplicate_titles)

title
Alone (2020)                                 4
The Box (2021)                               3
Journey to the Center of the Earth (2008)    3
The Plague (2006)                            3
Sweetheart (2019)                            2
                                            ..
Believer (2018)                              2
The Call (2020)                              2
Home (2008)                                  2
Swan Song (2021)                             2
The Father (2020)                            2
Name: count, Length: 202, dtype: int64


*kan tas sbort*

In [10]:
df_eda_mov["genres"]

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
                            ...                     
86532                                   Action|Drama
86533                                  Action|Horror
86534                                    Documentary
86535                                 Crime|Thriller
86536                                    Documentary
Name: genres, Length: 86537, dtype: object

In [11]:
# Dela upp genrerna och platta ut listorna till separata rader
genres_series = df_movies["genres"].str.split('|').explode()

# Visa unika genrer
unique_genres = genres_series.unique()
print(unique_genres)

# Eller räkna antalet filmer per genre
genre_counts = genres_series.value_counts()
print(genre_counts)


['Adventure' 'Animation' 'Children' 'Comedy' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Mystery' 'Sci-Fi' 'IMAX'
 'Documentary' 'War' 'Musical' 'Western' 'Film-Noir' '(no genres listed)']
genres
Drama                 33681
Comedy                22830
Thriller              11675
Romance               10172
Action                 9563
Documentary            9283
Horror                 8570
(no genres listed)     7060
Crime                  6917
Adventure              5349
Sci-Fi                 4850
Animation              4579
Children               4367
Mystery                3972
Fantasy                3821
War                    2301
Western                1690
Musical                1059
Film-Noir               354
IMAX                    195
Name: count, dtype: int64


### Dataset Ratings EDA:

In [12]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [13]:
num_ratngs = len(df_ratings)
num_movies = df_ratings["movieId"].nunique()
num_users = df_ratings["userId"].nunique()

print(f"Number of ratings: {num_ratngs}")
print(f"Number of unique movieId's: {num_movies}")
print(f"Number of unique users: {num_users}")
print(f"Average number of ratings per user: {round(num_ratngs/num_users), 2}")
print(f"Average number of ratings per movie: {round(num_ratngs/num_movies), 2}")

Number of ratings: 33832162
Number of unique movieId's: 83239
Number of unique users: 330975
Average number of ratings per user: (102, 2)
Average number of ratings per movie: (406, 2)


In [14]:
df_ratings.describe()

,userId,movieId,rating,timestamp
count,3.383216e+07,3.383216e+07,3.383216e+07,3.383216e+07
mean,1.654380e+05,2.831348e+04,3.542540e+00,1.269362e+09
std,9.534122e+04,4.992865e+04,1.063959e+00,2.541023e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,8.295300e+04,1.219000e+03,3.000000e+00,1.046718e+09
50%,1.661290e+05,3.263000e+03,4.000000e+00,1.264740e+09
75%,2.474500e+05,4.049100e+04,4.000000e+00,1.496919e+09
max,3.309750e+05,2.889830e+05,5.000000e+00,1.689843e+09


### Dataset Tags EDA:


In [15]:
df_tags.info()
df_tags

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328315 entries, 0 to 2328314
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   userId     int64 
 1   movieId    int64 
 2   tag        object
 3   timestamp  int64 
dtypes: int64(3), object(1)
memory usage: 71.1+ MB


,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746
...,...,...,...,...
2328310,330923,176599,politically correct,1507547491
2328311,330933,3317,coming of age,1351279384
2328312,330933,3317,sexuality,1351279389
2328313,330947,5782,Not Luc Besson,1154110902


In [16]:
# Undersök df_links
df_links.info() 
df_links.head()
df_links.describe()
df_links.columns
df_links.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  86537 non-null  int64  
 1   imdbId   86537 non-null  int64  
 2   tmdbId   86411 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.0 MB


,movieId,imdbId,tmdbId
count,86537.000000,8.653700e+04,8.641100e+04
mean,155932.817096,2.696384e+06,2.363101e+05
std,78037.145347,4.078415e+06,2.404829e+05
min,1.000000,1.000000e+00,2.000000e+00
25%,111443.000000,9.381100e+04,4.632800e+04
50%,164574.000000,4.845620e+05,1.362170e+05
75%,211225.000000,3.775202e+06,3.762850e+05
max,288983.000000,2.822749e+07,1.147612e+06


In [19]:
df_tags.describe()

,userId,movieId,timestamp
count,2.328315e+06,2.328315e+06,2.328315e+06
mean,1.776665e+05,7.055280e+04,1.522919e+09
std,8.285669e+04,7.402427e+04,1.308546e+08
min,1.000000e+01,1.000000e+00,1.135429e+09
25%,1.144050e+05,3.897000e+03,1.457997e+09
50%,2.154900e+05,5.107700e+04,1.561826e+09
75%,2.154900e+05,1.175330e+05,1.614286e+09
max,3.309470e+05,2.889550e+05,1.689841e+09


In [17]:
# Vill se hur måmnga taggar som finns per användare
df_tags["userId"].value_counts().head(10)
# Vill se hur många taggar som finns per film
#df_tags["movieId"].value_counts().head(10)
# Vill se hur många taggar som finns per film


userId
215490    723473
229729     20317
63350      19345
126357     17990
137062     16739
183281     15467
274127     15431
298708     13838
234824     13700
95828      12681
Name: count, dtype: int64

In [24]:

# 1) True/False: finns en specfik film bland titlarna?
def find_movie_title_partial(df_movies, partial_title):
    """
    Check if a movie title exists (even if misspelled or partial) and list all matching titles.
    """
    # Create a mask for titles containing the partial or misspelled word
    mask = df_movies["title"].str.contains(partial_title, case=False, na=False)
    
    # Check if any title matches the mask
    if mask.any():
        print(f"Movies containing '{partial_title}':")
        print(df_movies.loc[mask, "title"].unique())
        return mask
    else:
        print(f"No movies found containing '{partial_title}'.")
        return mask

# Example usage
mask = find_movie_title_partial(df_movies, "Star War")
print(df_movies.loc[mask, "title"].unique())

Movies containing 'Star War':
['Star Wars: Episode IV - A New Hope (1977)'
 'Star Wars: Episode V - The Empire Strikes Back (1980)'
 'Star Wars: Episode VI - Return of the Jedi (1983)'
 'Star Wars: Episode I - The Phantom Menace (1999)'
 'Star Wars: Episode II - Attack of the Clones (2002)'
 'Star Wars: Episode III - Revenge of the Sith (2005)'
 'Star Wars: The Clone Wars (2008)'
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy (2004)"
 'Star Wars: Threads of Destiny (2014)'
 'Star Wars: Episode VII - The Force Awakens (2015)'
 'The Star Wars Holiday Special (1978)' 'Robot Chicken: Star Wars (2007)'
 'Plastic Galaxy: The Story of Star Wars Toys (2014)'
 'Rogue One: A Star Wars Story (2016)' 'Star Wars: The Last Jedi (2017)'
 'Robot Chicken: Star Wars Episode II (2008)'
 'Robot Chicken: Star Wars Episode III (2010)'
 'Star Wars: The Legacy Revealed (2007)' 'Star Wars Downunder (2013)'
 'Solo: A Star Wars Story (2018)' 'Star Wars: Dresca'
 'Star Wars: The Rise of Skywalker (2019)

In [ ]:
def load_movies(path: str) -> pd.DataFrame:
    """
    Load and preprocess movies.csv:
     - expand ~
     - drop duplicate IDs
     - extract + remove (YYYY)
     - reorder 'X, The' → 'The X'
     - clean text
    """
    path = os.path.expanduser(path)
    df = pd.read_csv(path).drop_duplicates(subset="movieId")

    # 1) plocka ut år
    years = df["title"].str.extract(r"\((\d{4})\)")
    df["year"] = pd.to_numeric(years[0], errors="coerce").fillna(0).astype(int)

    # 2) ta bort '(YYYY)'  
    df["title"] = df["title"].str.replace(
        r"\s*\(\d{4}\)", "", regex=True
    )

    # 3) reorder 'X, The' → 'The X'
    df["title"] = df["title"].str.replace(
        r"^(.+),\s*(The|An|A)$", r"\2 \1", regex=True
    )

    # 4) städa texten
    df["title"] = df["title"].map(rensa_text)
    df["genres"] = df["genres"].map(lambda g: g.replace('|',' ')).map(rensa_text)

    return df